# Imports

In [ ]:
import sys
import copy
import numpy as np
import pandas as pd
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci, logger
from utils import BasicStatistics, RemovePolicy, Propaganda, setup_city_layout, confidence_interval
from model.utils import SocialPolicy
from model.debugutils import DebugUtils

# Parameters settings

Default parameters. Unless explicitly overriden, these values are used in all scenarios

In [ ]:
basic_parameters = SimulationParameters(
    hospitalization_capacity = 1.0,
    icu_capacity = 1.0
)

# Simulation setup

In [ ]:
population_size = 1000
simulation_cycles = 90 # days
multiple_runs = 20

## Setup of monitored variables

In [ ]:
basic_r0 = []
deaths_by_age_group = [[], [], [], [], [], [], [], [], [], []]
count_by_age_group = [[], [], [], [], [], [], [], [], [], []]
deaths_total = []

# Run simulation multiple times

In [ ]:
for k in range(multiple_runs):
    params = copy.deepcopy(basic_parameters)
    set_parameters(params)
    model = CovidModel()
    setup_city_layout(model, population_size)
    statistics = BasicStatistics(model)
    model.add_listener(statistics)
    debug = DebugUtils(model)
    logger().model = model
    for i in range(simulation_cycles):
        model.step()
    debug.update_infection_status()
    basic_r0.append(debug.get_R0_stats()['infections'].mean())
    stats = debug.get_age_group_stats()
    deaths_total.append(stats['deaths'].sum() / (stats['deaths'].sum() + stats['recovered'].sum()))
    for i in range(10):
        infected = stats['deaths'][i] + stats['recovered'][i]
        rate = 0 if infected == 0 else stats['deaths'][i] / infected
        deaths_by_age_group[i].append(rate)
        count_by_age_group[i].append(stats['count'][i])

# Print results

In [ ]:
print(confidence_interval(basic_r0))

In [ ]:
print(f"Total: {confidence_interval(deaths_total)}")
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(deaths_by_age_group[i])}")

In [ ]:
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(count_by_age_group[i])}")